In [1]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install -U google-cloud-storage {USER_FLAG} -q
! pip3 install {USER_FLAG} kfp google-cloud-pipeline-components --upgrade -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kfp 1.8.14 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.
google-cloud-pipeline-components 1.0.25 requires google-cloud-storage<2,>=1.20.0, but you have google-cloud-storage 2.5.0 which is incompatible.


## Restart the kernel

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
import kfp
from kfp.v2 import dsl
from kfp.v2.dsl import component
from kfp.v2.dsl import (
    Input,
    Output,
    Artifact,
    Dataset,
)

### Pipeline cloud parameters

In [2]:
#The Google Cloud project that this pipeline runs in.
project_id = "deassignement1"
# The region that this pipeline runs in
region = "us-central1"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
pipeline_root_path = "gs://temp-storage-group1/"

# Create Pipeline Components

#### Data Ingestion

In [3]:
from typing import Dict

def download_data(project_id: str, bucket: str, file_name: str) -> Dict:
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(bucket)
    blob = bucket.blob(file_name)
    local_path = '/tmp/'+ file_name
    blob.download_to_filename(local_path)
    logging.info('Downloaded Data!')
    
    # Convert the data to a dictiory object    
    dict_from_csv = pd.read_csv(local_path, index_col="Id", squeeze=True).to_dict()
    logging.info('Returning Data as Dictionary Object!')
    return dict_from_csv

In [4]:
# create a KFP component for data ingestion
data_ingestion_comp = kfp.components.create_component_from_func(
    download_data, output_component_file='data_ingestion.yaml', packages_to_install=['google-cloud-storage', 'pandas'])

#### Model Training

In [5]:
from typing import NamedTuple, Dict
def train_decision_tree (features: Dict, project_id: str, model_repo: str) -> Dict:
    '''train a Decision Tree with default parameters'''
    import pandas as pd
    from google.cloud import storage
    from sklearn.tree import DecisionTreeClassifier
    import json
    import logging 
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.DataFrame.from_dict(features)  
    
    logging.info(df.columns)
        
    # split into input (X) and output (Y) variables
    X = df.loc[:, ['SepalLengthCm','SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']].values
    y = df.loc[:, ['Species']].values
    
    # define model
    decision_tree_classifier = DecisionTreeClassifier(criterion = 'gini')
    decision_tree_classifier.fit(X, y)

    # evaluate the model
    score = decision_tree_classifier.score(X, y)
    logging.info("accuracy")
    metrics = {
        "accuracy": score,
    }
   
    # Save the model localy
    local_file = '/tmp/DecisionTree.pkl'
    joblib.dump(decision_tree_classifier, local_file)
     # write out output
  
    # Save to GCS as model.h5
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('DecisionTree.pkl')
    # Upload the locally saved model
    blob.upload_from_filename(local_file)

    print("Saved the model to GCP bucket : " + model_repo)
    return metrics

In [6]:
train_decision_tree_com = kfp.components.create_component_from_func(
    train_decision_tree, output_component_file='training_decision_tree.yaml', packages_to_install=['google-cloud-storage', 'pandas', 'joblib', 'scikit-learn'])

#### Prediction 

In [7]:
def predict_decision_tree(project_id: str, model_repo: str, features: Dict) -> Dict:
    import pandas as pd
    import joblib
    from google.cloud import storage
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df = pd.DataFrame.from_dict(features)    
    
    client = storage.Client(project=project_id)
    bucket = client.get_bucket(model_repo)
    blob = bucket.blob('DecisionTree.pkl')
    filename = '/tmp/DecisionTree.pkl'
    blob.download_to_filename(filename)
        
    #Loading the saved model with joblib
    model = joblib.load(filename)

    xNew = df[['SepalLengthCm','SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']]

    dfcp = df.copy()   
    y_classes = model.predict(xNew)
    logging.info(y_classes)
    dfcp['pclass'] = y_classes.tolist()
    dic = dfcp.to_dict(orient='records') 
    return dic

In [8]:
# create a KFP component for prediction LR 
prediction_decision_tree = kfp.components.create_component_from_func(
    predict_decision_tree, output_component_file='prediction_decision_tree.yaml', packages_to_install=['google-cloud-storage', 'pandas', 'joblib', 'scikit-learn'])

#### Define The Pipeline

In [9]:
@kfp.dsl.pipeline(
    name="iris-predictor-pipeline",
    pipeline_root=pipeline_root_path)
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str,): #testset_filename: str, ):
    
    
    di_op = data_ingestion_comp(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )

 
    training_dt_job_run_op = train_decision_tree_com(
        project_id=project_id,
        model_repo=model_repo,       
        features=di_op.output
    )
    
    """
    pre_di_op = data_ingestion_comp(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    ).after(training_mlp_job_run_op, training_lr_job_run_op)  
    
    # defining the branching condition
    with dsl.Condition(comp_model__op.output=="MLP"):
        predict_mlp_job_run_op = prediction_mpl_com(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.output
        )
    with dsl.Condition(comp_model__op.output=="LR"):
        predict_lr_job_run_op = prediction_lr_com(
        project_id=project_id,
        model_repo=model_repo,       
        features=pre_di_op.output
       )
       
       """

In [10]:
from kfp.v2 import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='iris_predictor_training_pipeline.json')

/home/jupyter/.local/lib/python3.7/site-packages/kfp/v2/compiler/compiler.py:1293: FutureWarning: APIs imported from the v1 namespace (e.g. kfp.dsl, kfp.components, etc) will not be supported by the v2 compiler since v2.0.0
  category=FutureWarning,


##### Run pipeline

In [ ]:
import google.cloud.aiplatform as aip

job = aip.PipelineJob(
    display_name="iris-predictor",
    enable_caching=False,
    template_path="iris_predictor_training_pipeline.json",
    pipeline_root=pipeline_root_path,
    parameter_values={
        'project_id': project_id, # makesure to use your project id 
        'data_bucket': 'data-group1',  # makesure to use your data bucket name 
        'trainset_filename': 'Iris.csv',     # makesure to upload these to your data bucket from DE2022/lab4/data
     #   'testset_filename': 'prediction_set.csv',    # makesure to upload these to your data bucket from DE2022/lab4/data
        'model_repo':'model-repository-group1' # makesure to use your model bucket name 
    }
)

job.run()